In [25]:
import numpy as np
import pandas as pd
import time 
import xgboost as xgb
from sklearn.cross_validation import train_test_split
from sklearn.preprocessing import OneHotEncoder

import matplotlib.pyplot as plt
import matplotlib as mpl
mpl.rcParams['font.sans-serif'] = ['SimHei']
mpl.rcParams['font.serif'] = ['SimHei']
import seaborn as sns
%matplotlib inline
p = sns.color_palette()
sns.set_style("darkgrid",{"font.sans-serif":['simhei', 'Arial']})

import warnings
warnings.filterwarnings('ignore')

**读入数据:**

# 用户信息表：  
69495 rows × 303 columns

In [42]:
train = pd.read_csv("../feature/训练表.csv",encoding="gb2312") 
test = pd.read_csv("../feature/测试表.csv",encoding="gb2312") 
test=test.fillna(-1)

dataset= pd.concat([train,test],axis=0)
trains = dataset.copy()


print(trains.shape)
trains.head()

(69495, 7)


,用户标识,标签,用户性别,用户职业,用户教育程度,用户婚姻状态,用户户口类型
0,1,0.0,1,2,3,1,3
1,2,0.0,1,2,3,2,1
2,3,0.0,1,4,4,1,4
3,4,1.0,1,4,4,3,2
4,5,0.0,1,2,2,3,1


In [43]:
# train = pd.read_csv("../feature/训练表.csv",encoding="gb2312") 
# test = pd.read_csv("../feature/测试表.csv",encoding="gb2312") 
# test=test.fillna(-1)

# dataset= pd.concat([train,test],axis=0)
# label=dataset[['用户标识','标签']]
# dataset=dataset.drop(['用户标识'],axis=1)

# #dataset=pd.get_dummies(dataset,columns=dataset[['用户性别','用户职业','用户教育程度','用户婚姻状态','用户户口类型']]).drop(['标签'],axis=1)
# #dataset #69495 rows × 24 columns
# # from sklearn.preprocessing import PolynomialFeatures
# # poly = PolynomialFeatures(degree=3,interaction_only=True,)#之前設置的是2，现在改成3，更多的交互特征
# # dataset=poly.fit_transform(dataset)
# # df = pd.DataFrame(dataset.reshape(69495,2325))
# # df.interpolate(axis=1).values.reshape(dataset.shape)

# #dataset
# #trains=pd.merge(label,df,left_index=True,right_index=True,how='outer')
# trains=pd.concat([label,dataset],axis=1)

# print(trains.shape)
# trains.head()#69495 rows × 303 columns

# 用户浏览行为训练表：  
69495 rows × 343 columns

In [44]:
train = pd.read_csv("../feature/用户浏览行为测试表.csv",encoding="gb2312") # 注意自己数据路径
test = pd.read_csv("../feature/用户浏览行为训练表.csv",encoding="gb2312") # 注意自己数据路径
dataset= pd.concat([train,test],axis=0)
temp=dataset[:]
#统计行和列的缺失值数
dataset=dataset.fillna(-1)
temp['用户浏览行为缺失统计']=(dataset==-1).sum(axis=1)
#用均值填充缺失值
temp.fillna(temp.mean(),inplace=True)
trains=pd.merge(trains,temp,how='left', on = "用户标识")

print(trains.shape)
trains.head()#69495 rows × 343columns

(69495, 47)


,用户标识,标签,用户性别,用户职业,用户教育程度,用户婚姻状态,用户户口类型,放款时间,放款前浏览行为数据min,放款前浏览行为数据mean,...,放款后浏览子行为编号_3,放款后浏览子行为编号_4,放款后浏览子行为编号_5,放款后浏览子行为编号_6,放款后浏览子行为编号_7,放款后浏览子行为编号_8,放款后浏览子行为编号_9,放款后浏览子行为编号_10,放款后浏览子行为编号_11,用户浏览行为缺失统计
0,1,0.0,1,2,3,1,3,68458,30.491576,109.746537,...,3.718407,70.073693,40.287381,36.067905,48.294558,15.810158,0.258406,32.14646,0.023401,38
1,2,0.0,1,2,3,2,1,68458,7.000000,116.091954,...,90.000000,150.000000,105.000000,135.000000,105.000000,105.000000,0.000000,75.00000,0.000000,8
2,3,0.0,1,4,4,1,4,68458,35.000000,112.824561,...,0.000000,48.000000,30.000000,24.000000,42.000000,0.000000,0.000000,24.00000,0.000000,8
3,4,1.0,1,4,4,3,2,68458,35.000000,106.494505,...,4.000000,64.000000,36.000000,32.000000,36.000000,4.000000,0.000000,24.00000,0.000000,8
4,5,0.0,1,2,2,3,1,68458,30.491576,109.746537,...,3.718407,70.073693,40.287381,36.067905,48.294558,15.810158,0.258406,32.14646,0.023401,38


# 用户银行流水记录表：  
69495 rows × 369  columns

In [45]:
train = pd.read_csv("../feature/用户银行流水记录训练表.csv",encoding="gb2312") 
test = pd.read_csv("../feature/用户银行流水记录测试表.csv",encoding="gb2312")
dataset= pd.concat([train,test],axis=0)
dataset = dataset.drop(['放款时间'],axis=1)
temp=dataset.copy()
#统计行和列的缺失值数
dataset=dataset.fillna(-1)
#temp['用户银行流水记录缺失统计']=(dataset==-1).sum(axis=1)
#用均值填充缺失值
temp.fillna(temp.mean(),inplace=True)
trains=pd.merge(trains,temp,how='left', on = "用户标识")

print(trains.shape)
trains.head()#69495 rows × 369  columns

(69495, 71)


,用户标识,标签,用户性别,用户职业,用户教育程度,用户婚姻状态,用户户口类型,放款时间,放款前浏览行为数据min,放款前浏览行为数据mean,...,放款后用户支出总计,放款后用户支出笔数,放款后用户工资收入笔数,放款后用户工资收入总计,放款后用户收入支出笔数差值,放款后用户收入支出总计差值,放款后用户非工资收入笔数,放款后用户非工资收入总计,放款后工资收入笔数乘以差值,放款后工资收入总计乘以差值
0,1,0.0,1,2,3,1,3,68458,30.491576,109.746537,...,1358.128024,116.241542,4.851931,64.717081,-71.873742,-776.353235,68.791845,891.918534,-821.802575,-118239.074919
1,2,0.0,1,2,3,2,1,68458,7.000000,116.091954,...,1358.128024,116.241542,4.851931,64.717081,-71.873742,-776.353235,68.791845,891.918534,-821.802575,-118239.074919
2,3,0.0,1,4,4,1,4,68458,35.000000,112.824561,...,1358.128024,116.241542,4.851931,64.717081,-71.873742,-776.353235,68.791845,891.918534,-821.802575,-118239.074919
3,4,1.0,1,4,4,3,2,68458,35.000000,106.494505,...,1358.128024,116.241542,4.851931,64.717081,-71.873742,-776.353235,68.791845,891.918534,-821.802575,-118239.074919
4,5,0.0,1,2,2,3,1,68458,30.491576,109.746537,...,1358.128024,116.241542,4.851931,64.717081,-71.873742,-776.353235,68.791845,891.918534,-821.802575,-118239.074919


# 用户账单表初级特征:  
69495 rows × 448  columns

In [46]:
train = pd.read_csv("../feature/用户账单表初级特征训练表.csv",encoding="gb2312") 
test = pd.read_csv("../feature/用户账单表初级特征测试表.csv",encoding="gb2312") 
dataset= pd.concat([train,test],axis=0)
dataset = dataset.drop(['放款时间'],axis=1)
temp=dataset.copy()

#统计行和列的缺失值数
dataset=dataset.fillna(-1)
#temp['用户银行流水记录缺失统计']=(dataset==-1).sum(axis=1)
#特征轮：构造放款后与放款前差值特征
feature_name=temp.columns 
for name in feature_name:
    if name.find('款后')> 0:
        x1=name.replace('款后','款前')
        if x1 in feature_name:
            temp[name+"与"+x1+"差值"]=temp[name]-temp[x1]        
#用均值填充缺失值
temp.fillna(temp.mean(),inplace=True)
#temp

trains=pd.merge(trains,temp,how='left', on = "用户标识")


print(trains.shape)
trains.head()#69495 rows × 448  columns

(69495, 149)


,用户标识,标签,用户性别,用户职业,用户教育程度,用户婚姻状态,用户户口类型,放款时间,放款前浏览行为数据min,放款前浏览行为数据mean,...,放款后账单金额为零与放款前账单金额为零差值,放款后账单金额统计与放款前账单金额统计差值,放款后还款金额为负数与放款前还款金额为负数差值,放款后还款金额为零与放款前还款金额为零差值,放款后还款金额统计与放款前还款金额统计差值,用户放款后信用卡额度最大值与用户放款前信用卡额度最大值差值,用户放款后循环利息与用户放款前循环利息差值,用户放款后循环利息(去重)与用户放款前循环利息(去重)差值,用户放款后消费笔数与用户放款前消费笔数差值,用户放款后消费笔数(去重)与用户放款前消费笔数(去重)差值
0,1,0.0,1,2,3,1,3,68458,30.491576,109.746537,...,-9.188051,-541.365080,-0.143285,-10.227881,-521.889947,-0.103084,-82.248417,-44.04394,-36.130726,-20.255175
1,2,0.0,1,2,3,2,1,68458,7.000000,116.091954,...,1.000000,52.433636,0.000000,1.000000,45.794699,1.609437,0.000000,0.00000,11.000000,11.000000
2,3,0.0,1,4,4,1,4,68458,35.000000,112.824561,...,-9.188051,-541.365080,-0.143285,-10.227881,-521.889947,-0.103084,-82.248417,-44.04394,-36.130726,-20.255175
3,4,1.0,1,4,4,3,2,68458,35.000000,106.494505,...,-2.000000,-27.976233,0.000000,-2.000000,-32.142947,-0.154151,0.000000,0.00000,0.000000,0.000000
4,5,0.0,1,2,2,3,1,68458,30.491576,109.746537,...,-9.188051,-541.365080,-0.143285,-10.227881,-521.889947,-0.103084,-82.248417,-44.04394,-36.130726,-20.255175


# 用户账单表特征:  
69495 rows × 1146 columns


In [47]:
train = pd.read_csv("../feature/用户账单表特征训练表_整体.csv",encoding="gb2312") 
test = pd.read_csv("../feature/用户账单表特征测试表_整体.csv",encoding="gb2312") 
dataset= pd.concat([train,test],axis=0)

temp=dataset.copy()
#print(temp.shape)#(69495, 466)
#统计行和列的缺失值数
dataset=dataset.fillna(-1)
temp['用户银行流水记录缺失统计']=(dataset==-1).sum(axis=1)
#特征轮：构造放款后与放款前差值特征
feature_name=temp.columns 
for name in feature_name:
    if name.find('款后')> 0:
        x1=name.replace('款后','款前')
        if x1 in feature_name:
            temp[name+"与"+x1+"差值"]=temp[name]-temp[x1]        
#用均值填充缺失值
temp.fillna(temp.mean(),inplace=True)

trains=pd.merge(trains,temp,how='left', on = "用户标识")

print(trains.shape)
trains.head()#69495 rows × 1146 columns

(69495, 390)


,用户标识,标签,用户性别,用户职业,用户教育程度,用户婚姻状态,用户户口类型,放款时间,放款前浏览行为数据min,放款前浏览行为数据mean,...,去重后整体本期账单金额var,去重后整体调整金额var,去重后整体循环利息var,去重后整体可用余额var,去重后整体预借现金额度var,去重后整体上期还款金额var与整体上期账单金额var差值,去重后整体信用卡额度var与整体本期账单余额var差值,去重后整体可用余额var与整体预借现金额度var差值,去重后整体本期账单最低还款额var与整体循环利息var之和,用户银行流水记录缺失统计
0,1,0.0,1,2,3,1,3,68458,30.491576,109.746537,...,48.035133,5.203795,15.743922,6.407825,52.971646,-15.529225,10.390762,-46.563821,30.291245,240
1,2,0.0,1,2,3,2,1,68458,7.000000,116.091954,...,71.470209,0.000000,0.000000,0.000000,98.469100,-2.824033,32.567287,-98.469100,145.318143,0
2,3,0.0,1,4,4,1,4,68458,35.000000,112.824561,...,48.035133,5.203795,15.743922,6.407825,52.971646,-15.529225,10.390762,-46.563821,30.291245,15
3,4,1.0,1,4,4,3,2,68458,35.000000,106.494505,...,78.000994,0.000000,0.000000,18.103521,87.803786,-20.167102,62.381889,-69.700264,0.954005,0
4,5,0.0,1,2,2,3,1,68458,30.491576,109.746537,...,48.035133,5.203795,15.743922,6.407825,52.971646,-15.529225,10.390762,-46.563821,30.291245,15


In [48]:
trains.to_csv("../feature/特征汇总.csv",index=None,encoding="gb2312")